In [6]:
import os

In [10]:
%pwd

'd:\\_MLOPS_project\\end-to-end-MLOPS-project-by-atishay'

In [9]:
os.chdir('../')

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [12]:
from  mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [16]:
class ConfigerationManager:
    def __init__(    
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAM_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params  = read_yaml(param_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])           
    
        data_ingestion_config = DataIngestionConfig(       
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [17]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [18]:
class DataInestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )    
            logger.info(f"{filename} download with the following info: \n{headers}")
        else:
            logger.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        etract the file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [20]:
#### PIPELINE
try:
    config = ConfigerationManager()
    data_ingestion_config =  config.get_data_ingestion_config()
    data_ingestion = DataInestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-03-17 16:57:59,735:INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-03-17 16:57:59,735:INFO: common: yaml file:params.yaml loaded successfully]
[2024-03-17 16:57:59,750:INFO: common: yaml file:schema.yaml loaded successfully]
[2024-03-17 16:57:59,752:INFO: common: created directories at:artifacts]
[2024-03-17 16:57:59,752:INFO: common: created directories at:artifacts/data_ingestion]
[2024-03-17 16:58:00,613:INFO: 2257987178: artifacts/data_ingestion/data.zip download with the following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "884c1cf387c0bdb6970fcaa7b4cb24e4f8c4f6c0175daae87349b3272c500902"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 37F6:182048:11C416:1931E5:65F6D3C0
Accept-Ranges: bytes
Date: Sun, 17 Mar